In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import tensorflow_io as tfio
import soundfile as sf
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from functools import partial

from data import fetch_metadata,load_wav_for_map, load_wav_16k_mono, bird_mapping_idx_to_name
from yamnet import Yamnet

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
assert len(tf.config.list_physical_devices('GPU')) >=1 

In [3]:
yamnet_base = Yamnet()
yamnet_class_names = yamnet_base.yamnet_class_names

In [4]:
birdsong_metadata = fetch_metadata()
dataset = birdsong_metadata.copy()

train_audio_paths, valid_audio_paths, train_labels, valid_labels = train_test_split(
    dataset[["filepath"]], dataset["label"], stratify=dataset["label"], test_size=0.2, random_state=101
)

filepaths = pd.concat([train_audio_paths, valid_audio_paths])
folds = pd.Series([0] * len(train_audio_paths) + [1] * len(valid_audio_paths))
targets = pd.concat([train_labels, valid_labels])

main_ds = tf.data.Dataset.from_tensor_slices((filepaths.values.squeeze(), targets.values.squeeze(), folds.values.squeeze()))
main_ds = tf.data.Dataset.from_tensor_slices((filepaths.values.squeeze(), targets.values.squeeze(), folds.values.squeeze()))
main_ds = main_ds.map(load_wav_for_map)

main_ds = main_ds.map(yamnet_base.extract_embedding).unbatch()
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold ==0)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold ==1)

remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
# def load_wav_16k_mono(filename):
#     """ read in a waveform file and convert to 16 kHz mono """
#     file_contents = tf.io.read_file(filename)
#     wav, sample_rate = tf.audio.decode_wav(
#         file_contents,
#         desired_channels=1)
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
#     return wav

# shapes = []
# for f in tqdm(filepaths.values.squeeze()):
#     try:
#         tensor = load_wav_16k_mono(f)
#         shapes.append(tensor.shape[0])
#     except Exception as e:
#         print(e)
#         print(f)
#         print('error!!')

 44%|████▎     | 8360/19114 [28:26<25:39,  6.98it/s]  

unable to initialize resampler: 3 [Op:IO>AudioResample]
/media/wwymak/Storage2/birdsong_dataset/xeno_canto_eu_cleaned/willow_warbler/465780.wav
error!!


 70%|██████▉   | 13335/19114 [45:10<08:27, 11.39it/s]  

unable to initialize resampler: 3 [Op:IO>AudioResample]
/media/wwymak/Storage2/birdsong_dataset/xeno_canto_eu_cleaned/common_reed_bunting/459261.wav
error!!


 92%|█████████▏| 17548/19114 [1:00:28<03:51,  6.77it/s]

unable to initialize resampler: 3 [Op:IO>AudioResample]
/media/wwymak/Storage2/birdsong_dataset/xeno_canto_eu_cleaned/common_chiffchaff/465779.wav
error!!


 98%|█████████▊| 18813/19114 [1:04:51<01:22,  3.63it/s]

unable to initialize resampler: 3 [Op:IO>AudioResample]
/media/wwymak/Storage2/birdsong_dataset/xeno_canto_eu_cleaned/dunnock/560551.wav
error!!


100%|██████████| 19114/19114 [1:05:48<00:00,  4.84it/s]


In [5]:
ym_finetune_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(birdsong_metadata.label.nunique())
], name='ym_finetune_model')

ym_finetune_model.summary()

Model: "ym_finetune_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 84)                21588     
Total params: 677,716
Trainable params: 677,716
Non-trainable params: 0
_________________________________________________________________


In [6]:
metrics = [
    tf.keras.metrics.SparseTopKCategoricalAccuracy(
        k=5, name="sparse_top_k_categorical_accuracy", dtype=None
    ),
    'accuracy'
]
optimiser = tf.keras.optimizers.Adam(
    learning_rate=5e-4,
)
loss = loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
ym_finetune_model.compile(loss=loss,
                 optimizer=optimiser,
                 metrics=metrics)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="test_model_all_birds",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="./logs",
        update_freq="batch",
    )  # How often to write logs (default: once per epoch)
]
history = ym_finetune_model.fit(train_ds,
                       epochs=30,
                       validation_data=val_ds,
                       callbacks=callbacks)

Epoch 1/30
      1/Unknown - 0s 188us/step - loss: 4.7027 - sparse_top_k_categorical_accuracy: 0.0625 - accuracy: 0.0000e+00WARNING:tensorflow:From /home/wwymak/anaconda3/envs/birdsong_recognition/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


  80955/Unknown - 9932s 123ms/step - loss: 2.1994 - sparse_top_k_categorical_accuracy: 0.7117 - accuracy: 0.4727
Epoch 00001: val_loss improved from inf to 4.28632, saving model to test_model_all_birds
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

80967/80967 [==============================] - 116s 1ms/step - loss: 2.5712 - sparse_top_k_categorical_accuracy: 0.6208 - accuracy: 0.3751 - val_loss: 3.8769 - val_sparse_top_k_categorical_accuracy: 0.4266 - val_accuracy: 0.1688
Epoch 4/30
80935/80967 [============================>.] - ETA: 0s - loss: 2.5650 - sparse_top_k_categorical_accuracy: 0.6222 - accuracy: 0.3743
Epoch 00004: val_loss improved from 3.87687 to 3.77116, saving model to test_model_all_birds
INFO:tensorflow:Assets written to: test_model_all_birds/assets


INFO:tensorflow:Assets written to: test_model_all_birds/assets


80967/80967 [==============================] - 117s 1ms/step - loss: 2.5649 - sparse_top_k_categorical_accuracy: 0.6222 - accuracy: 0.3744 - val_loss: 3.7712 - val_sparse_top_k_categorical_accuracy: 0.4344 - val_accuracy: 0.1722
Epoch 5/30
80935/80967 [============================>.] - ETA: 0s - loss: 2.5479 - sparse_top_k_categorical_accuracy: 0.6256 - accuracy: 0.3784
Epoch 00005: val_loss did not improve from 3.77116
80967/80967 [==============================] - 116s 1ms/step - loss: 2.5476 - sparse_top_k_categorical_accuracy: 0.6257 - accuracy: 0.3785 - val_loss: 3.8960 - val_sparse_top_k_categorical_accuracy: 0.4135 - val_accuracy: 0.1635
Epoch 6/30
80935/80967 [============================>.] - ETA: 0s - loss: 2.5295 - sparse_top_k_categorical_accuracy: 0.6284 - accuracy: 0.3825
Epoch 00006: val_loss did not improve from 3.77116
80967/80967 [==============================] - 117s 1ms/step - loss: 2.5292 - sparse_top_k_categorical_accuracy: 0.6285 - accuracy: 0.3825 - val_loss: 3

INFO:tensorflow:Assets written to: test_model_all_birds/assets


80967/80967 [==============================] - 116s 1ms/step - loss: 2.5162 - sparse_top_k_categorical_accuracy: 0.6336 - accuracy: 0.3868 - val_loss: 3.7280 - val_sparse_top_k_categorical_accuracy: 0.4354 - val_accuracy: 0.1753
Epoch 17/30
80935/80967 [============================>.] - ETA: 0s - loss: 2.5184 - sparse_top_k_categorical_accuracy: 0.6325 - accuracy: 0.3842
Epoch 00017: val_loss did not improve from 3.72799
80967/80967 [==============================] - 116s 1ms/step - loss: 2.5182 - sparse_top_k_categorical_accuracy: 0.6326 - accuracy: 0.3843 - val_loss: 3.8180 - val_sparse_top_k_categorical_accuracy: 0.4409 - val_accuracy: 0.1816
Epoch 18/30
80935/80967 [============================>.] - ETA: 0s - loss: 2.5198 - sparse_top_k_categorical_accuracy: 0.6339 - accuracy: 0.3832
Epoch 00018: val_loss did not improve from 3.72799
80967/80967 [==============================] - 116s 1ms/step - loss: 2.5195 - sparse_top_k_categorical_accuracy: 0.6339 - accuracy: 0.3833 - val_loss:

In [7]:
ym_finetune_model.save('/media/wwymak/Storage2/birdsong_dataset/models/ym_finetune_baseline')

INFO:tensorflow:Assets written to: /media/wwymak/Storage2/birdsong_dataset/models/ym_finetune_baseline/assets


INFO:tensorflow:Assets written to: /media/wwymak/Storage2/birdsong_dataset/models/ym_finetune_baseline/assets


In [3]:
class ReduceMeanLayer(tf.keras.layers.Layer):
    def __init__(self, axis=0, **kwargs):
        super(ReduceMeanLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, input):
        return tf.math.reduce_mean(input, axis=self.axis)

In [4]:
class ReduceMeanLayer(tf.keras.layers.Layer):
    
    def call(self, input):
        return tf.math.reduce_mean(input, axis=0)
ym_finetune_model =  tf.keras.models.load_model('models/test_model_all_birds')


saved_model_path = './models/model_all_birds_v1.h5'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer(yamnet_base.model_path,
                                            trainable=False, name='yamnet')
_, embeddings_output, _ = embedding_extraction_layer(input_segment)
serving_outputs = ym_finetune_model(embeddings_output)
serving_outputs = ReduceMeanLayer(name='classifier')(serving_outputs)
# serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)

In [5]:
import gc
del serving_model
gc.collect()

8138

In [11]:

reloaded_model = tf.keras.models.load_model(saved_model_path, custom_objects={'ReduceMeanLayer': ReduceMeanLayer, 'KerasLayer': hub.KerasLayer})

In [12]:
testing_wav_data = load_wav_16k_mono('/media/wwymak/Storage2/birdsong_dataset/xeno_canto_eu_cleaned/willow_ptarmigan/139115.wav')
results = reloaded_model(testing_wav_data)
# birds = yamnet_class_names[tf.math.top_k(results,k=5)]

In [20]:
Audio(testing_wav_data, rate=16000)

In [14]:
top_probs, top_idxs = tf.math.top_k(results, k=5)
_, most_likely_idx = tf.math.top_k(results, k=1)
top_labels = [bird_mapping_idx_to_name[idx] for idx in top_idxs.numpy()]
top_labels

['common_blackbird',
 'european_green_woodpecker',
 'common_nightingale',
 'willow_ptarmigan',
 'tawny_owl']

In [16]:
bird_mapping_idx_to_name[most_likely_idx.numpy()[0]]

'common_blackbird'

In [18]:
tf.math.top_k??

Signature: tf.math.top_k(input, k=1, sorted=True, name=None)
Source:   
@tf_export("math.top_k", "nn.top_k")
@dispatch.add_dispatch_support
def top_k(input, k=1, sorted=True, name=None):  # pylint: disable=redefined-builtin
  """Finds values and indices of the `k` largest entries for the last dimension.

  If the input is a vector (rank=1), finds the `k` largest entries in the vector
  and outputs their values and indices as vectors.  Thus `values[j]` is the
  `j`-th largest entry in `input`, and its index is `indices[j]`.

  For matrices (resp. higher rank input), computes the top `k` entries in each
  row (resp. vector along the last dimension).  Thus,

      values.shape = indices.shape = input.shape[:-1] + [k]

  If two elements are equal, the lower-index element appears first.

  Args:
    input: 1-D or higher `Tensor` with last dimension at least `k`.
    k: 0-D `int32` `Tensor`.  Number of top elements to look for along the last
      dimension (along each row for matrices).
   

In [22]:
sorted(yamnet_class_names)

['A capella',
 'Accelerating, revving, vroom',
 'Accordion',
 'Acoustic guitar',
 'Afrobeat',
 'Air brake',
 'Air conditioning',
 'Air horn, truck horn',
 'Aircraft',
 'Aircraft engine',
 'Alarm',
 'Alarm clock',
 'Ambient music',
 'Ambulance (siren)',
 'Angry music',
 'Animal',
 'Applause',
 'Arrow',
 'Artillery fire',
 'Babbling',
 'Baby cry, infant cry',
 'Baby laughter',
 'Background music',
 'Bagpipes',
 'Bang',
 'Banjo',
 'Bark',
 'Basketball bounce',
 'Bass drum',
 'Bass guitar',
 'Bathtub (filling or washing)',
 'Beatboxing',
 'Bee, wasp, etc.',
 'Beep, bleep',
 'Bell',
 'Bellow',
 'Belly laugh',
 'Bicycle',
 'Bicycle bell',
 'Bird',
 'Bird flight, flapping wings',
 'Bird vocalization, bird call, bird song',
 'Biting',
 'Bleat',
 'Blender',
 'Bluegrass',
 'Blues',
 'Boat, Water vehicle',
 'Boiling',
 'Boing',
 'Boom',
 'Bouncing',
 'Bow-wow',
 'Bowed string instrument',
 'Brass instrument',
 'Breaking',
 'Breathing',
 'Burping, eructation',
 'Burst, pop',
 'Bus',
 'Busy signal'